# ECDSA 

Алгоритм цифровой подписи с использованием эллиптических кривых 

Допустим Алиса хочет подписать свое сообщение m и отправить его Бобу.
Они договариваются о кривой $E(F_p)$ и точке на ней $G$, $ord(G) = q$. 

Алиса выбирает приватный ключ $d_a \in [1, q-1]$. $Q_a = d_a * G$.

Для дальнейшей подписи Алиса выполняет несколько шагов:

1. e = HASH(m)
2. $z = F_p(e)$
3. Случайно выбирается $k \in [1, q-1]$
4. $R = k * G$, $r = R.x$
5. $s = k^{-1} * (z + r * d_a) \pmod{q}$
6. $(r, s)$ - цифровая подпись сообщения m.


Проверка:

1. $u_1 = z * s^{-1} \pmod{q}$
2. $u_2 = r * s^{-1} \pmod{q}$
3. $(u_1 * G + u_2 * Q_a).x == r$


In [10]:
from hashlib import blake2b

p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
e = EllipticCurve(GF(p), [0, 3])
G = e.random_element()
q = e.order()
assert G.order() == e.order()
d_a = randint(1, q-1)
Q_a = d_a * G

o = G.order()
m = b"Where's your motivation?"

def sign(m):
    z = int(blake2b(m, digest_size=q.bit_length() // 16).hexdigest(), 16)
    k = randint(1, q-1)
    r = int((k * G)[0]) % o
    s = pow(int(k), -1, q) * (z + int(r) * d_a) % q
    return r, s, z


def verify(G, Q, r, s, o):
    u1 = pow(s, -1, o) * z % o
    u2 = pow(s, -1, o) * r % o

    assert int((u1 * G + u2 * Q)[0]) % o == r

r, s, z = sign(m)

verify(G, Q_a, r, s, o)

# Nonce Reuse 

Допустим, кто-то не послушал условия в предыдущей секции и решил два раза использовать один и тот же nonce(k). 
Это можно понять, например, если $r_1 = r_2$ для каких то двух сообщений. 

тогда 

$s_1 = k_1^{-1} * (z_1 + r_1 * d_a) \pmod{ord(G)}$

$s_2 = k_2^{-1} * (z_2 + r_2 * d_a) \pmod{ord(G)}$

$s_1 - s_2 = k^{-1}(z_1 - z_2) \pmod{ord(G)}$

$k = (z_1 - z_2) * (s_1 - s_2)^{-1} \pmod{ord(G)}$

In [16]:
k = randint(1, q-1)

In [21]:
m1 = b"Put your Curve Away Walter"
m2 = b"Heisenburger, please"

def sign(m):
    z = int(blake2b(m, digest_size=q.bit_length() // 16).hexdigest(), 16)
    r = int((k * G)[0]) % o
    s = pow(int(k), -1, q) * (z + int(r) * d_a) % q
    return r, s, z

sig1 = sign(m1)
sig2 = sign(m2)

r1, s1, z1 = sig1
r2, s2, z2 = sig2

k = (z1 - z2) * pow(s1 - s2, -1, o) % o

d = (s1 * k - z1) * pow(r1, -1, o) % o
d

16119135902351571520453442811871580027364205679996006764158118558377836933618

In [22]:
Q_a, d * G

((3280766721065071630939891882847820472644694453927095347044796964490400873472 : 123810441720935794545967070096899493176168878996421771814820203418245191862 : 1),
 (3280766721065071630939891882847820472644694453927095347044796964490400873472 : 123810441720935794545967070096899493176168878996421771814820203418245191862 : 1))

# Linear/Polynomial Congruential Generator

Представтье себе, что на сервере, который генерирует подписи, для создания случайных чисел $k$ используют рекуррентную зависимость линейную или с помощью многочленов:

LCG: $x_{n} = a * x_{n-1} + b \pmod{p}$, сид - $x_0$

Линейный: $x_{n} = a_1 * x_{n - 1} + a_2 * x_{n-2} + ... + a_k * x_{n - k} \pmod{p}$, сид - $x_0, x_1, ..., x_{k-1}$ 

Многочлены: $x_{n} = a_k * x_{n-1}^k + a_{k-1} * x_{n-1}^{k-1} + ... + a_1 * x_{n-1} + a_0 \pmod{p}$, сид - $x_0$



Теперь рассмотрим две подписи:

$s_0 = k_0^{-1} * (h_0 + r_0 * d) \pmod{q}$

$s_1 = k_1^{-1} * (h_1 + r_1 * d) \pmod{q}$

$k_0 = \frac{h_0}{s_0} + \frac{r_0}{s_0} * d$

$k_1 = \frac{h_1}{s_1} + \frac{r_1}{s_1} * d$

После избавления от $d$ и некоторого преобразования получим зависимость:

$k_1 = \frac{r_1 * s_0}{r_0 * s_1} * k_0 + \frac{h_1 * r_0 - h_0 * r_1}{r_0 * s_1}$

Заметьте, что не знаем мы тут только $k_1, k_0$...

Из этого, кстати, следует, что мы можем решить задачу дискретного логарифма для $k_1 * G -  \frac{r_1 * s_0}{r_0 * s_1} * k_0 * G = R_1 + a * R_2$, что в целом не очень хорошо, но и не критично. 

In [4]:
m1 = b"Put your Curve Away Walter"
m2 = b"Heisenburger, please"

def sign_leak(m):
    z = int(blake2b(m, digest_size=q.bit_length() // 16).hexdigest(), 16)
    k = randint(1, q-1)
    r = (k * G)[0]
    s = pow(int(k), -1, q) * (z + int(r) * d_a) % q
    return int(r), int(s), int(k)
    
r1, s1, k1 = sign_leak(m1)
r2, s2, k2 = sign_leak(m2)
h1 = int(blake2b(m1, digest_size=q.bit_length() // 16).hexdigest(), 16)
h2 = int(blake2b(m2, digest_size=q.bit_length() // 16).hexdigest(), 16)

assert (k1 * s1) % q == (h1 + r1 * d_a) % q
a = ((r2 * s1 * k1) * pow(r1 * s2, -1, q) + (h2 * r1 - h1 * r2) * pow(r1 * s2, -1, q)) % q

a == k2

True

А критично сдесь следующее:

Для $N$ выбранных подписей мы можем составить систему уравнений:

- Случай многочленов

$k_1 = a_t * k_0^t + a_{t-1} * k_{0}^{t-1} + ... + a_1 * k_{0} + a_0 \pmod{q}$

$k_2 = a_t * k_1^t + a_{t-1} * k_{1}^{t-1} + ... + a_1 * k_{1} + a_0 \pmod{q}$

...

$k_N = a_t * k_{N-1}^t + a_{t-1} * k_{N-1}^{t-1} + ... + a_1 * k_{N-1} + a_0 \pmod{q}$

И с учетом полученной выше зависимости мы можем дополнить эту систему еще $N - 1$ уравнением

Вместо каждого $k_i$ подставим $\frac{h_0}{s_0} + \frac{r_0}{s_0} * d$ и решим систему линейных уравнений относительно $a_i$ в зависимости от переменной $d$. 
Далее мы получаем многочлен степени $t$ от $d$ и его уже решить довольно просто. 

In [24]:
class poly_rel:
    def __init__(self, seed, n, q, cfs=None):
        self.state = seed
        x = var('x')
        P = PolynomialRing(GF(q), x) 
        if cfs is None:
            self.poly = P.random_element(degree=n)
        else:
            assert len(cfs) == n
            self.poly = P(cfs)
            
    def next(self):
        self.state = self.poly(x=self.state)
        return self.state

generator = poly_rel(randint(0, q), 3, q)

def pro_sign(m):
    z = int(blake2b(m, digest_size=q.bit_length() // 16).hexdigest(), 16)
    k = generator.next()
    r = (k * G)[0]
    s = pow(int(k), -1, q) * (z + int(r) * d_a) % q
    return int(r), int(s)

In [25]:
from os import urandom
t = 4
ms = [urandom(17) for _ in range(t + 2)]
hs = [int(blake2b(m, digest_size=q.bit_length() // 16).hexdigest(), 16) for m in ms]
sigs = [pro_sign(m) for m in ms]

triplets = [(h, s[0], s[1]) for h, s in zip(hs, sigs)] 

In [26]:
d = var('d')
P = PolynomialRing(GF(q), d)
d = P(d)

ks = [P(hi  + ri * d) * pow(si, -1, q) for hi, ri, si in triplets]

M = Matrix(P, [[pow(k, i) for i in range(t)] for k in ks[:-2]])
V = vector(P, ks[1:-1])
T = M.solve_right(V)
den = M.det()

final_poly = P(ks[-1] * den - den * sum(T[i] * ks[-2]**i for i in range(t)))
final_poly.roots()

[(16119135902351571520453442811871580027364205679996006764158118558377836933618,
  1)]

In [28]:
d_a

16119135902351571520453442811871580027364205679996006764158118558377836933618